# Оглавление

* [Обучение модели](#Обучение-модели)
* [Оценка качества поиска](#Оценка-качества-поиска)
    * [по УДК](#по-УДК)
    * [по ГРНТИ](#по-ГРНТИ)

# Обучение модели

In [5]:
import sys
sys.path.append('../../ap/utils')
from model_train import fit_topic_model

In [ ]:
fit_topic_model()

In [9]:
!pwd

/home/tishchenko_e/text_categorization/research/notebooks


# Оценка качества поиска

In [15]:
config_search_quality = {
    'path_model': '/data/antiplagiat_models/best_model/models/topic_model',
    'path_results': '/data/antiplagiat_models/best_model/results',
    'path_experiment': '/home/tishchenko_e/text_categorization/best_model/experiments',
    'bcg_topic_list': ['topic_0'],
    'artm_model_params': {
        'num_collection_passes': 25,
        'NUM_TOPICS': 125,
        'num_bcg_topic': 1,
        'tau_DecorrelatorPhi': 0.05,
        'tau_SmoothTheta': 0.025,
        'tau_SparseTheta': -0.1,
    },
    'metrics_to_calculate': ['analogy'],
    'recalculate_train_centroids': True,
    'recalculate_test_thetas': True,
    'path_train_thetas': '/data/datasets/Antiplagiat/combined_train/train_by_lang'
}

In [16]:
import sys
sys.path.append('../../ap/utils')
from search_quality import calculate_search_quality

In [17]:
search_quality = calculate_search_quality(config_search_quality)

  0%|          | 0/15 [00:00<?, ?it/s]

Calculation of train centroids were started.


  7%|▋         | 1/15 [04:11<58:42, 251.58s/it]

 13%|█▎        | 2/15 [11:21<1:06:05, 305.05s/it]

 20%|██        | 3/15 [12:07<45:26, 227.24s/it]  

...

{'average_frequency_analogy': 0.9560717488789238, 'average_percent_analogy': 0.33749327354260095}


  0%|          | 0/1 [00:00<?, ?it/s]

Thetas of test data were calculated.


100%|██████████| 1/1 [00:01<00:00,  1.44s/it]


{'average_frequency_analogy': 1.0, 'average_percent_analogy': 0.326}


In [19]:
search_quality

{'Средняя частота УДК': 0.96762331838565,
 'Средний процент УДК': 0.21321973094170407,
 'Средняя частота ГРНТИ': 0.9560717488789238,
 'Средний процент ГРНТИ': 0.33749327354260095,
 'Средняя частота ВАК': 1.0,
 'Средний процент ВАК': 0.326}

# Оценка качества классификации

### по УДК

In [1]:
import sys
sys.path.append('../../ap/utils')
from pathlib import Path
import config
from sklearn.linear_model import LogisticRegression

In [2]:
sys.path.append('../../research/experimental_code')
from Classification import GetAccuracy

In [3]:
log_reg_classifier = LogisticRegression(random_state=42, solver='saga', penalty='l1', C=0.1)

/data/antiplagiat/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
current_lang = 'ru'
config_classification_quality = {
    'path_model': '/data/antiplagiat_models/best_model/models/topic_model',
    'path_test': str(Path(config.path_articles_test_bpe).joinpath(f'test_{current_lang}.txt')),
    'path_rubrics_train': config.path_articles_rubrics_train_udk,
    'path_rubrics_test': config.path_articles_rubrics_train_udk,
    'path_train': '/data/antiplagiat_models/best_model/data/batches_balanced',
    'recalculate_train_theta': False,
    'path_to_save_train_theta': '/data/datasets/Antiplagiat/train_rubrics_modalities/',
    'classifier': log_reg_classifier
}

In [5]:
classificator_udk = GetAccuracy(config_classification_quality)

In [6]:
accuracy_udk,  *_ = classificator_udk.get_accuracy()

In [12]:
accuracy_udk

0.5148078134845621

### по ГРНТИ

In [8]:
config_classification_quality['path_rubrics_train'] = '/data/datasets/Antiplagiat/grnti_codes_numbers_without_no.json'
config_classification_quality['path_rubrics_test'] = '/data/datasets/Antiplagiat/grnti_codes_numbers_without_no.json'

In [9]:
classificator_grnti = GetAccuracy(config_classification_quality)

In [10]:
accuracy_grnti,  *_ = classificator_grnti.get_accuracy()

In [13]:
accuracy_grnti

0.23125393824826718